In [1]:
!pip install pandas scikit-learn flask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.6/226.6 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.2.3
    Uninstalling Werkzeug-2.2.3:
      Successfully uninstalled Werkzeug-2.2.3
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 3.0.3
    Uninstalling Jinja2-3.0.3:
      Successfully uninstalled Jinja2-3.0.3
  Attempting uninstall: click
    Found existing installation: click 8.0.4
    Uninstalling click-8.0.4:
      Successfully uninstalled click-8.0.4
  Attempting uninstall: blinker
    Found existing installation: blinker 1.4
    Uninstalling blinker-1.4:
      Successfully uninstalled blinker-1.4
ERROR: pip's dependency resolver does not currently take into acco

In [2]:
# Import necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from flask import Flask, request, jsonify

In [3]:
# Load your dataset
df = pd.read_csv('https://raw.githubusercontent.com/yashwantaditya009/Data_Science_Projects/main/Data%20Science%20Projects%20Certification/Text%20Recommendation%20System/Dataset/flipkart_com-ecommerce_sample.csv')
df

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,7179d2f6c4ad50a17d014ca1d2815156,2015-12-01 10:15:43 +0000,http://www.flipkart.com/walldesign-small-vinyl...,WallDesign Small Vinyl Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE7KFJAKSTDY9G,1500.0,730.0,"[""http://img6a.flixcart.com/image/wall-decorat...",False,Buy WallDesign Small Vinyl Sticker for Rs.730 ...,No rating available,No rating available,WallDesign,"{""product_specification""=>[{""key""=>""Number of ..."
19996,71ac419198359d37b8fe5e3fffdfee09,2015-12-01 10:15:43 +0000,http://www.flipkart.com/wallmantra-large-vinyl...,Wallmantra Large Vinyl Stickers Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE9F5URNQGJCGH,1429.0,1143.0,"[""http://img6a.flixcart.com/image/sticker/z/g/...",False,Buy Wallmantra Large Vinyl Stickers Sticker fo...,No rating available,No rating available,Wallmantra,"{""product_specification""=>[{""key""=>""Number of ..."
19997,93e9d343837400ce0d7980874ece471c,2015-12-01 10:15:43 +0000,http://www.flipkart.com/elite-collection-mediu...,Elite Collection Medium Acrylic Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE7VAYDKQZEBSD,1299.0,999.0,"[""http://img5a.flixcart.com/image/sticker/b/s/...",False,Buy Elite Collection Medium Acrylic Sticker fo...,No rating available,No rating available,Elite Collection,"{""product_specification""=>[{""key""=>""Number of ..."
19998,669e79b8fa5d9ae020841c0c97d5e935,2015-12-01 10:15:43 +0000,http://www.flipkart.com/elite-collection-mediu...,Elite Collection Medium Acrylic Sticker,"[""Baby Care >>

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  20000 non-null  object 
 1   crawl_timestamp          20000 non-null  object 
 2   product_url              20000 non-null  object 
 3   product_name             20000 non-null  object 
 4   product_category_tree    20000 non-null  object 
 5   pid                      20000 non-null  object 
 6   retail_price             19922 non-null  float64
 7   discounted_price         19922 non-null  float64
 8   image                    19997 non-null  object 
 9   is_FK_Advantage_product  20000 non-null  bool   
 10  description              19998 non-null  object 
 11  product_rating           20000 non-null  object 
 12  overall_rating           20000 non-null  object 
 13  brand                    14136 non-null  object 
 14  product_specifications

In [5]:
product_name = df['product_name']
product_name

0            Alisha Solid Women's Cycling Shorts
1            FabHomeDecor Fabric Double Sofa Bed
2                                     AW Bellies
3            Alisha Solid Women's Cycling Shorts
4          Sicons All Purpose Arnica Dog Shampoo
                          ...                   
19995             WallDesign Small Vinyl Sticker
19996    Wallmantra Large Vinyl Stickers Sticker
19997    Elite Collection Medium Acrylic Sticker
19998    Elite Collection Medium Acrylic Sticker
19999    Elite Collection Medium Acrylic Sticker
Name: product_name, Length: 20000, dtype: object

In [6]:
hp_products = df[df['product_name'].str.startswith("HP")]
hp_products

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
7948,650e27dff8f645649353743f64e03f77,2016-03-11 11:26:55 +0000,http://www.flipkart.com/hp-v220w-16-gb-pen-dri...,HP v220w 16 GB Pen Drive,"[""Computers >> Storage >> Pen Drives >> HP Pen...",ACCEGF6WWRAUMRPT,750.0,550.0,"[""http://img5a.flixcart.com/image/pendrive/r/p...",False,Specifications of HP v220w 16 GB Pen Drive (S...,No rating available,No rating available,HP,"{""product_specification""=>[{""key""=>""Interface""..."
8271,88ca277594f22bb7494f2d0bbc51f8ba,2016-01-07 05:50:25 +0000,http://www.flipkart.com/hp-pavilion-dv7t-6100-...,HP Pavilion DV7t-6100 Series 6 Cell Laptop Bat...,"[""Computers >> Laptop Accessories >> Batteries...",ACCDSY8EGJPW7RJB,4999.0,4999.0,"[""http://img6a.flixcart.com/image/laptop-batte...",False,Buy HP Pavilion DV7t-6100 Series 6 Cell Laptop...,No rating available,No rating available,HP,"{""product_specification""=>[{""key""=>""In The Box..."
8454,aa3fbcc0c0dfcc7ce64eedd347e7fab4,2016-01-07 05:50:25 +0000,http://www.flipkart.com/hp-pavilion-dv6-2059eo...,HP Pavilion DV6-2059eo 6 Cell Laptop Battery,"[""Computers >> Laptop Accessories >> Batteries...",ACCDSY8E3XGY8UFZ,4984.0,4984.0,"[""http://img5a.flixcart.com/image/laptop-batte...",False,Buy HP Pavilion DV6-2059eo 6 Cell Laptop Batte...,No rating available,No rating available,HP,"{""product_specification""=>[{""key""=>""In The Box..."
8455,f314ef29ad9dd30b019586615be04030,2016-01-07 05:50:25 +0000,http://www.flipkart.com/hp-pavilion-dv6z-6-cel...,HP Pavilion DV6z 6 Cell Laptop Battery,"[""Computers >> Laptop Accessories >> Batteries...",ACCDSY8EYJ6PTNGT,4999.0,4999.0,"[""http://img5a.flixcart.com/image/laptop-batte...",False,Buy HP Pavilion DV6z 6 Cell Laptop Battery onl...,No rating available,No rating available,HP,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
8456,77f659d4d18381c991fb9210e05d9c1b,2016-01-07 05:50:25 +0000,http://www.flipkart.com/hp-pavilion-dv6-2070eo...,HP Pavilion DV6-2070eo 6 Cell Laptop Battery,"[""Computers >> Laptop Accessories >> Batteries...",ACCDSY8EYXYYYYVK,4999.0,4999.0,"[""http://img5a.flixcart.com/image/laptop-batte...",False,Buy HP Pavilion DV6-2070eo 6 Cell Laptop Batte...,No rating available,No rating available,HP,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
8457,0aa757eaaa5055e2ac3e51db494ea8c4,2016-01-07 05:50:25 +0000,http://www.flipkart.com/hp-presario-g50-6-cell...,HP Presario G50 6 Cell Laptop Battery,"[""Computers >> Laptop Accessories >> Batteries...",ACCDSY8ERUYRYSUA,4999.0,3690.0,"[""http://img5a.flixcart.com/image/laptop-batte...",False,Buy HP Presario G50 6 Cell Laptop Battery only...,No rating available,No rating available,HP,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
8464,689c167e8186dd88294366cb129d3d40,2016-01-07 05:50:25 +0000,http://www.flipkart.com/hp-pavilion-g71-6-cell...,HP Pavilion G71 6 Cell Laptop Battery,"[""Computers >> Laptop Accessories >> Batteries...",ACCDSY8EDTGSEZD5,4999.0,3515.0,"[""http://img5a.flixcart.com/image/laptop-batte...",False,HP Pavilion G71 6 Cell Laptop Battery\n ...,No rating available,No rating available,HP,"{""product_specification""=>[{""key""=>""In The Box..."
8467,66b17d68e8773683cf77c7d79c20d890,2016-01-07 05:50:25 +0000,http://www.flipkart.com/hp-paviliondv5-1019tx-...,HP PavilionDV5-1019tx 6 Cell Laptop Battery,"[""Computers >> Laptop Accessories >> Batteries...",ACCDSY8EAED5TGZY,4999.0,4999.0,"[""http://img5a.flixcart.com/image/laptop-batte...",False,Buy HP PavilionDV5-1019tx 6 Cell Laptop Batter...,No rating available,No rating available,HP,"{""product_specification""=>[{""key""=>""In The Box..."
8469,51add18129a213c1fffea43b69f8eb79,2016-01-07 05:50:25 +0000,http://www.flipkart.com/hp-pavilion-dv6-2060so...,HP Pavilion DV6-2060so 6 Cell Laptop Battery,"[""Computers >> Laptop Accessories >> Batteries...",ACC

In [7]:
hp_products.describe()

,retail_price,discounted_price
count,14.000000,14.000000
mean,8973.285714,8550.714286
std,12040.157720,12241.936646
min,750.000000,248.000000
25%,4973.500000,3558.750000
50%,4999.000000,4991.500000
75%,4999.000000,4999.000000
max,38890.000000,38890.000000


In [8]:
hp_products.count()

uniq_id                    14
crawl_timestamp            14
product_url                14
product_name               14
product_category_tree      14
pid                        14
retail_price               14
discounted_price           14
image                      14
is_FK_Advantage_product    14
description                14
product_rating             14
overall_rating             14
brand                      13
product_specifications     14
dtype: int64

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['product_name'])


In [10]:
# Define the user's input
user_input = "HP Laptop"  # Replace with the user's query

# Calculate cosine similarities
cosine_similarities = cosine_similarity(tfidf_vectorizer.transform([user_input]), tfidf_matrix)

# Reshape the similarity scores to match the length of the DataFrame
cosine_similarities = cosine_similarities[0]

# Create a DataFrame to store product recommendations
recommendations = df[['product_name']].copy()
recommendations['similarity_score'] = cosine_similarities

# Sort by similarity score in descending order
recommendations = recommendations.sort_values(by='similarity_score', ascending=False)

# Display the top N recommendations
top_n = 3
top_recommendations = recommendations.head(top_n)
print("Top Recommendations:")
print(top_recommendations)


Top Recommendations:
                                product_name  similarity_score
8464   HP Pavilion G71 6 Cell Laptop Battery          0.501651
8455  HP Pavilion DV6z 6 Cell Laptop Battery          0.501651
8457   HP Presario G50 6 Cell Laptop Battery          0.481706
